# Monte Carlo Methods I: Random Numbers and Random Walk

Computational physics often begins with the notion that classical equations can perfectly predict the evolution of a system if one knows the initial conditions precisely.
However, many real-world processes involve an inherent level of randomness or produce outcomes so sensitive to initial conditions that they effectively appear random.
Radioactive decay, governed by quantum mechanics, is an example of truly non-deterministic behavior, whereas chaotic classical systems like the three-body problem can exhibit sensitive dependence on initial conditions.
As a result, introducing randomness into our computational methods is not only natural for modeling genuinely stochastic systems but also highly advantageous even in deterministic settings.

Monte Carlo methods, which rely on random sampling to estimate quantities of interest, illustrate how such randomness can be leveraged as a powerful tool in numerical analysis.
The phrase "Monte Carlo" alludes to the famous casino district in Monaco, highlighting the essence of "gambling" with random draws to tackle challenging problems.
Since their popularization in mid-twentieth-century nuclear research, Monte Carlo algorithms have found widespread use in evaluating high-dimensional integrals, simulating physical processes in statistical mechanics, performing error analysis in data-driven fields, and more.

![Monte Carlo](https://upload.wikimedia.org/wikipedia/commons/1/1b/Monaco_pano.jpg)

Although real physical systems may exhibit genuine randomness (as seen in quantum events), computer algorithms generate numbers through deterministic procedures.
These "pseudo-random" sequences are designed to mimic statistical randomness but are entirely reproducible given a specific initial seed.
In practice, a well-crafted pseudo-random number generator can satisfy the statistical requirements of scientific simulations, especially if it has a sufficiently large period and low correlations across many dimensions. Modern generators such as the Mersenne Twister, PCG, and Xorshift provide excellent performance in these regards, far surpassing the simpler linear congruential generators of older programming standards.

## Pseudo-Random Number Generators (PRNGs)

In computational physics, random draws are often produced by deterministic algorithms called pseudo-random number generators.
These generators start from an internal seed and use a mathematical rule to transform it repeatedly, producing an output sequence that ideally appears random.
Although each sequence is entirely predictable if one knows the seed and the algorithm, a good generator will avoid obvious patterns and pass stringent statistical tests.

Early implementations of pseudo-random number generators were often based on linear congruential generators. The simplest form of this approach looks something like
\begin{align}
x_{n+1} = (a x_n + c) \mathrm{\ mod\ } m,
\end{align}
where $x_n$ represents the state at step $n$, and $a$, $c$, and $m$ are carefully chosen integers.

In [ ]:
# In order to understand the concept of a random number generator, let's implement one ourself.

mynext = 1

def myrand(): # NOT RECOMMENDED for real application.
    global mynext # by default, python does not recognize variable in a different scope.
    mynext = mynext * 1103515245 + 12345
    return (mynext//65536) % 32768

# This random number generator would generate integers in the domain [0, 32768).
# This random is usually provided to user by

MYRAND_MAX = 32768-1

# There are reasons for choosing the strange constants.  Take a look at
#     https://en.wikipedia.org/wiki/Linear_congruential_generator
# if you are interested.

In [ ]:
# Now, every time we run `rand()`, we will get a different number

myrand()

In [ ]:
# We may just print many of them at the same time:

Rs = [myrand() for i in range(100)]
print(Rs)

In [ ]:
# Sometime it is useful to make sure your random number sequence remains the same.
# In our case, you may notice that we can simply reset the `mynext` global variable to reset the sequence.
# The value you put in `mynext` is often called the "seed".

print('The following two lists are not the same:')
print([myrand() for i in range(10)])
print([myrand() for i in range(10)])

print('We may ensure that they are the same by "seeding" the random number generator with a fixed value:')

mynext = 1234
print([myrand() for i in range(10)])

mynext = 1234
print([myrand() for i in range(10)])

In [ ]:
# We may even plot the random numbers

from matplotlib import pyplot as plt

ll = [[myrand() for i in range(100)] for j in range(100)]
plt.imshow(ll)

This random number generator is very simple and is the *sample* implementation in many ANSI C libraries!
However, because how the standard was written, this create a lot problems.
* The standard only require `RAND_MAX` be at least 32767.
  If one want to evulate 1e6 points (which is pretty small, as we will see below), you will actually be evaluating the same 32768 points 30 times each!
* Some implementation "tried" to imporve the algorithm, e.g., swapping the lower and higher bytes.
  But these tricks sometime ruins the generator!
* Integrating high-dimension space is an important application of Monte Carlo methods.
  However, the congruential generators create correlation in k-space.

When programming in Python, we often take advantage of the built-in random module or NumPy's random subpackage, both of which default to high-quality generators (the Mersenne Twister for Python's default, and similarly robust methods in NumPy).
We can seed these generators manually to ensure reproducible research, or we may allow them to seed automatically for convenience.
While these generators generally suffice for undergraduate-level simulations, more intricate studies---particularly those involving very large datasets or extremely sensitive measurements---may require advanced or specialized libraries for best results.

In [ ]:
# Let's now try python's random number library

import random as rnd

print(rnd.random())             # return a random float in the range [0,1)
print(rnd.randrange(100))       # return a random int in the range [0, stop)
print(rnd.randint(a=0,b=99))    # return a random int in the range [a, b+1)
print(rnd.gauss(mu=0, sigma=1)) # sample from a Gaussian distribution

In [ ]:
# We may plot the results of these random number generators

Rs = [rnd.random() for i in range(1000)]
plt.hist(Rs)

In [ ]:
Rs = [rnd.randrange(100) for i in range(1000)]
plt.hist(Rs)

In [ ]:
Rs = [rnd.gauss(0, 1) for i in range(1000)]
plt.hist(Rs)

In [ ]:
# There is also a seed() function

rnd.seed(1234)
print([rnd.randrange(100) for i in range(10)])

rnd.seed(1234)
print([rnd.randrange(100) for i in range(10)])